# Параллельные вычисления

Материалы:
* Макрушин С.В. Лекция 10: Параллельные вычисления
* https://docs.python.org/3/library/multiprocessing.html

## Задачи для совместного разбора

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


1. Посчитайте, сколько раз встречается каждый из символов (заглавные и строчные символы не различаются) в файле `Dostoevskiy Fedor. Prestuplenie i nakazanie - BooksCafe.Net.txt` и в файле `Dostoevskiy Fedor. Igrok - BooksCafe.Net.txt`. 

2. Решить задачу 1, распараллелив вычисления с помощью модуля `multiprocessing`. Для обработки каждого файла создать свой собственный процесс. 

## Лабораторная работа 10

In [ ]:
import multiprocessing as mp
from multiprocessing import Pool, Queue
import csv
from itertools import islice

1. Разбейте файл `recipes_full.csv` на несколько (например, 8) примерно одинаковых по объему файлов c названиями `id_tag_nsteps_*.csv`. Каждый файл содержит 3 столбца: `id`, `tag` и `n_steps`, разделенных символом `;`. Для разбора строк используйте `csv.reader`.

__Важно__: вы не можете загружать в память весь файл сразу. Посмотреть на первые несколько строк файла вы можете, написав код, который считывает эти строки.

Подсказка: примерное кол-во строк в файле - 2.3 млн.

Фрагмент одного из файлов, которые должны получиться в результате:
```
id;tag;n_steps
137739;60-minutes-or-less;11
137739;time-to-make;11
137739;course;11
```


In [ ]:

def count_rows():
  rows = 0
  with open('/content/drive/MyDrive/Colab Notebooks/lab 10/recipes_full.csv', newline='') as fok:
      reader = csv.reader(fok, delimiter=',')
      for row in reader:
          rows += len(row[5][2:-2].split("', '"))
  return rows


def making_file(count, fok, sizing, rows, header):
  Listach=[]
  for row in rows:
      if count >= sizing * fok:
          fok += 1
          current_out_writer = csv.writer(open(f"./id_tag_nsteps_{fok}.csv", 'w'), delimiter=";")
          current_out_writer.writerow(header)
          Listach.append(f"id_tag_nsteps_{fok}.csv")
      for tag in row[5][2:-2].split("', '"):
          count += 1
          current_out_writer.writerow([row[1], tag, row[6]])   
  return count,fok, Listach

def make_files(count,fok):
  sizing = count_rows()/8
  with open('/content/drive/MyDrive/Colab Notebooks/lab 10/recipes_full.csv', newline='') as csvfile:
    rows = csv.reader(csvfile, delimiter=',')
    titles = next(rows)
    header = [titles[1], titles[5][:-1], titles[6]]
    count,fok, Listach=making_file(count, fok, sizing, rows, header)
  return Listach


Listach=make_files(0,0)



2. Напишите функцию, которая принимает на вход название файла, созданного в результате решения задачи 1, считает среднее значение количества шагов для каждого тэга и возвращает результат в виде словаря.

In [ ]:
def calculate(rows, header):
  MeAn = {}
  for row in rows:
    if (row[1] in MeAn.keys()):
      MeAn[row[1]][0] += int(row[2])
      MeAn[row[1]][1] += 1
      else:
        MeAn[row[1]] = [int(row[2]), 1]
  return MeAn

def next_calc(MeAn):
  for idx, vals in MeAn.items():
    MeAn.update({idx : (vals[0] / vals[1])})
  return MeAn

def meanFile(): 
  with open('./id_tag_nsteps_1.csv',  "r+") as csvfile:
    rows = csv.reader(csvfile, delimiter=';')
    MeAn=calculate(rows, next(rows))
  return next_calc(MeAn)

meanFile()

3. Напишите функцию, которая считает среднее значение количества шагов для каждого тэга по всем файлам, полученным в задаче 1, и возвращает результат в виде словаря. Не используйте параллельных вычислений. При реализации выделите функцию, которая объединяет результаты обработки отдельных файлов. Модифицируйте код из задачи 2 таким образом, чтобы получить результат, имея результаты обработки отдельных файлов. Определите, за какое время задача решается для всех файлов.


In [ ]:
def calculattion(dictat,result):
  for d in dictat[1:]:
    for id, ar in d.items():
      if (id in result.keys()):
        result[id][0] += int(ar[0])
        result[id][1] += 1
  return result

def dic(filelist, output):
    dictat = []
    for file in filelist:
      dictat.append(MeAn(file))
    result = dictat[0]
    result=calculattion(dictat,result)
    for idx, vals in result.items():
        result.update({idx : (vals[0] / vals[1])})
    return result


def MeAn(file):    
  with open('./' + file,  "r+") as csvfile:
    rows = csv.reader(csvfile, delimiter=';')
    header = next(rows)
    for row in rows:
      if (row[1] in output.keys()):
        output[row[1]][0] += int(row[2])
        output[row[1]][1] += 1
      else:
        output[row[1]] = [int(row[2]), 1]
  return output

output = {} 
dic(Listach, output)


{'': 451.59124568043444,
 '1-day-or-more': 582.8234007654456,
 '15-minutes-or-less': 647.5982112587378,
 '3-steps-or-less': 604.4340311861353,
 '30-minutes-or-less': 978.6183264072014,
 '4-hours-or-less': 1283.2958792701008,
 '5-ingredients-or-less': 684.8088340848441,
 '60-minutes-or-less': 1203.5642053925365,
 'Throw the ultimate fiesta with this sopaipillas recipe from Food.com.': 451.04826203913706,
 'a1-sauce': 448.701978724577,
 'african': 562.2578220334847,
 'american': 966.9343406649201,
 'amish-mennonite': 465.38361904761905,
 'angolan': 444.12177228351914,
 'appetizers': 802.3029127231733,
 'apples': 625.4106585751528,
 'april-fools-day': 448.53359836563413,
 'argentine': 456.0781500082786,
 'artichoke': 447.798040192659,
 'asian': 825.376437405077,
 'asparagus': 521.1396748048331,
 'australian': 546.5548325813346,
 'austrian': 456.01455748552524,
 'avocado': 462.8856027706008,
 'bacon': 530.8434272414295,
 'baja': 460.49123598038665,
 'baked-beans': 445.79432037496554,
 'bak

4. Решите задачу 3, распараллелив вычисления с помощью модуля `multiprocessing`. Для обработки каждого файла создайте свой собственный процесс. Определите, за какое время задача решается для всех файлов.

In [ ]:
def calc (rows,ruch):
  pas = next(rows)
  for row in rows:
    if (row[1] in ruch.keys()):
      ruch[row[1]][0] += int(row[2])
      ruch[row[1]][1] += 1
    else:
      ruch[row[1]] = [int(row[2]), 1]
  return ruch

def means(filename):
    with open('./' + filename,  "r+") as fok:
      rows = csv.reader(fok, delimiter=';')
      return calc(rows,{}) 

def stepping(listes, result):
  for dct in listes[1:]:
    for id, oval in dct.items():
      if id in result.keys():
        result[id][0] += int(oval[0])
        result[id][1] += 1
  return result

def Fin(listes):
    result = listes[0]
    stepping(listes, result)
    for id, dar in result.items():
      result.update({id : (dar[0] / dar[1])})
    return result


def launch(Listach):
  %%time
  with mp.Pool(processes=8) as pool:
    output = pool.map(means, Listach)
    return (Fin(output))

launch(Listach)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 4.53 µs


{'': 28.035886214442012,
 '1-day-or-more': 35.94089264173703,
 '15-minutes-or-less': 40.166472038406646,
 '3-steps-or-less': 36.91372694640426,
 '30-minutes-or-less': 60.3274048367856,
 '4-hours-or-less': 80.24991158788164,
 '5-ingredients-or-less': 42.29552809649897,
 '60-minutes-or-less': 74.4680679785331,
 'Throw the ultimate fiesta with this sopaipillas recipe from Food.com.': 28.392148213498015,
 'a1-sauce': 28.495364238410595,
 'african': 34.891205458680815,
 'american': 59.28359872611465,
 'amish-mennonite': 28.36513443191674,
 'angolan': 27.366435385949696,
 'appetizers': 49.84430512016719,
 'apples': 38.271794871794874,
 'april-fools-day': 27.249241439098395,
 'argentine': 28.931081081081082,
 'artichoke': 27.357448630136986,
 'asian': 52.19441598360656,
 'asparagus': 32.66067048242028,
 'australian': 33.810364683301344,
 'austrian': 29.008,
 'avocado': 28.51352549889135,
 'bacon': 32.914,
 'baja': 28.25242718446602,
 'baked-beans': 27.92471213463242,
 'baking': 29.55706760316

5. Решите задачу 3, распараллелив вычисления с помощью модуля `multiprocessing`. Создайте фиксированное количество процессов (равное половине количества ядер на компьютере). При помощи очереди `multiprocessing.queue` передайте названия файлов для обработки процессам и при помощи другой очереди заберите от них ответы. 

In [ ]:


def calc(rows,ave):
  for row in rows:
    if row[1] in ave.keys():
      ave[row[1]][0] += int(row[2])
      ave[row[1]][1] += 1
    else:
      ave[row[1]] = [int(row[2]), 1]
  return ave

def aveMaria(filename):
  with open('./' + filename,  "r+") as csvfile:
    rows = csv.reader(csvfile, delimiter=';')
    pas = next(rows)
    return calc(rows,{})

def fork(Enter, Out):
  while True:
    m = Enter.get()
    if m == 'q':
      break
    else:
      Out.put(aveMaria(m))

def launch():
  %%time
  Enter = Queue()
  Out = Queue()
  p = Pool(2, f, (Enter, Out))
  listes = []
  for i in Listach:
    Enter.put(i)
  for i in range(len(Listach)):
    m = Out.get()
    listes.append(len(Listach))
  for i in range(2):
    Enter.put('q')
  Fin(listes)

launch()